In [41]:
import urllib.request
import numpy as np
import json
from sklearn import metrics as sk_m
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [2]:
# URL to retrive pre-trained 300 dimensional gloVe embedding
embedding_300_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors.txt"

def read_embedding(url):
    """Function to read out an embedding
    Input: url: url to embedding
    
    Returns: vocab: list of words in the embedding
             word2id: dictionary mapping words to ids
             word_vectors: array storing the embeddings,
                           row corresponds to word id"""
    # Open url
    data = urllib.request.urlopen(url)
    vocab = []
    word_vectors = []
    
    # Each line contains one word and its embedding
    for line in data:
        line = line.decode()
        # Split by spaces
        split = line.split()
        # First element(== the word) is added to vocabulary
        vocab.append(split[0])
        # All other elements(embedding vectors) are added to vectors
        word_vectors.append([float(elem) for elem in split[1:]])
    
    # Create a dictionary with word-id pairs based on the order
    word2id = {w: i for i, w in enumerate(vocab)}
    # Vectors are converted into an array
    word_vectors = np.array(word_vectors).astype(float)
    
    return vocab, word2id, word_vectors
    
embedding_300_vocab, embedding_300_word2id, embedding_300_word_vector = read_embedding(embedding_300_url)

In [3]:
def exclude_vocab(vocab, exclude):
    """Function to exclude specific words from vocabulary
    Input: vocab: list of words in the embedding
           exclude: list of words to exclude from the vocabulary
           
    Returns: limited_vocab: vocab without the words in exclude"""
    # Create copies of vocab, word2id and word_vector
    limited_vocab = vocab.copy()
    # For all words that are in exclude and vocab
    for word in exclude:
        if word in limited_vocab:
            # Remove word from vocab
            limited_vocab.remove(word)
            
    return limited_vocab

In [4]:
# URL to female specific words
female_words_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/female_word_file.txt"
female_words_data = urllib.request.urlopen(female_words_url)

# List of female words
female_words = []
for line in female_words_data:
    line = line.decode()
    line = line.split()
    female_words.append(line[0])

In [5]:
# URL to male specific words
male_words_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/male_word_file.txt"
male_words_data = urllib.request.urlopen(male_words_url)

# List of male words
male_words = []
for line in male_words_data:
    line = line.decode()
    line = line.split()
    male_words.append(line[0])

In [6]:
# Create List with female - male pairs from female-male specific words
female_male_pairs = []
for i, female in enumerate(female_words):
    female_male_pairs.append([female, male_words[i]])

In [7]:
# URLs to the files storing gender specific words
gender_specific_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/gender_specific_full.json"

# Empty list to accumulate gender specific words plus additional list after lowercasing
gender_specific_original = []
gender_specific = []


# Read out URL and add further gender specific words
with urllib.request.urlopen(gender_specific_url) as f:
    gender_specific_original.extend(json.load(f))

# Add lower case words to second list
for word in gender_specific_original:
    gender_specific.append(word.lower())

In [8]:
# URL to the file storing definitional pairs
definitial_pairs_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/definitional_pairs.json"

# Empty list to store definitional pairs plus additional list after lowercasing
definitional_pairs_original = []
definitional_pairs = []


# Read out url and add pairs in list
with urllib.request.urlopen(definitial_pairs_url) as f:
    definitional_pairs_original.extend(json.load(f))
    
# Add lower case pairs to second list
for [w1, w2] in definitional_pairs_original:
    definitional_pairs.append([w1.lower(), w2.lower()])


# Create list of single words instead of pairs  
definitional_words = []
for pair in definitional_pairs:
    for word in pair:
        definitional_words.append(word)
        

In [9]:
# URL to the file storing the equalize pairs
equalize_pairs_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/equalize_pairs.json"

# Empty list to store equalize pairs plus additional list after lowercasing
equalize_pairs_original = []
equalize_pairs = []

# Read out URL and add pairs to list
with urllib.request.urlopen(equalize_pairs_url) as f:
    equalize_pairs_original.extend(json.load(f))
    
# Add lower case pairs to second list
for [w1, w2] in equalize_pairs_original:
    equalize_pairs.append([w1.lower(), w2.lower()])
    
# Create list of single words instead of pairs
equalize_words = []
for pair in equalize_pairs:
    for word in pair:
        equalize_words.append(word)

In [10]:
# List of all gender specific words included in 
# female words, male words, gender specific words, equalize words and definitional words
exclude_words = list(set(female_words + male_words + gender_specific + definitional_words + equalize_words))

In [11]:
# Remove gender specific words from the embedding to obtain vocabulary of neutral words
embedding_300_vocab_neutral = exclude_vocab(embedding_300_vocab, exclude_words)
print("vocab size: ", len(embedding_300_vocab))
print("limited vocab size: ", len(embedding_300_vocab_neutral))

vocab size:  322636
limited vocab size:  321977


In [12]:
def idtfy_gender_subspace(W, w2id, definitional_pairs, embedding, k=1):
    C = np.ndarray((10, 2, 300))
    for i, d_pair in enumerate(definitional_pairs):
        for j, word in enumerate(d_pair):
            mean_i = embedding[w2id[word]] / len(d_pair) # should be 2
            C[i][j] = ((np.transpose(embedding[w2id[word]] - mean_i)*(embedding[w2id[word]] - mean_i))/len(d_pair))
    C = C.reshape(10, 300, 2)
    
    
    # is C supposed to be the covariance matrix of the word embeddings?
    array = np.ndarray((10,2,300))
    #i=0
    array_two = np.zeros((10,300,300))
    for j, d_pair in enumerate(definitional_pairs):
        for i, word in enumerate(d_pair):
            # fill array with embeddings
            array[j][i] = embedding[w2id[word]]
            #i = i+1
        # print(array[j][0].shape)
        # calculate covariance between embeddings of same definitional pair?
        array_two[j]=np.cov(np.transpose(array[j]))
        
    # print(array_two.shape)
    #new_C = np.cov(array)
    #print(np.shape(new_C))
    _, new_SVD_C, _ = np.linalg.svd(array_two)
    print(np.shape(new_SVD_C))
    new_B = new_SVD_C[:k]
    print(new_B.shape)
    
    _, SVD_C, _ = np.linalg.svd(C, full_matrices = True)
    # print(np.shape(SVD_C))
    B = SVD_C[:k]
    return new_B

In [13]:
B = idtfy_gender_subspace(embedding_300_vocab, embedding_300_word2id, definitional_pairs, embedding_300_word_vector)
print(B)

(10, 300)
(1, 300)
[[1.22104626e+01 5.10738137e-15 3.71802466e-15 3.35278652e-15
  2.68827222e-15 2.51849119e-15 2.40515216e-15 2.07717457e-15
  1.61296100e-15 1.30848799e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22005497e-15
  1.22005497e-15 1.22005497e-15 1.22005497e-15 1.22

In [35]:
# most biased male and female words
def most_biased(embedding, B, k=500):
    # small x, else memory issues
    x = 50000
    all_biased = np.ndarray((x,1))
    for i, word in enumerate(embedding):
        if i < x:
            all_biased[i] = (sk_m.pairwise.cosine_similarity(word.reshape(1,300), B))[0]
            # print(sk_m.pairwise.cosine_similarity(word.reshape(1,300), B)[0])
    #print(all_biased)
    most_biased_m = []
    most_biased_f = []
    for word in range(k):
        # male words
        mb_index = np.argmax(all_biased)
        most_biased_m.append(mb_index)
        all_biased[mb_index] = 0
        # female words
        fb_index = np.argmin(all_biased)
        most_biased_f.append(fb_index)
        all_biased[fb_index] = 0
    #print(most_biased_m, most_biased_f)
    return most_biased_m, most_biased_f

In [ ]:
index_m, index_f = most_biased(embedding_300_word_vector, B)
for i in index_m:
    print(embedding_300_vocab[i])

In [ ]:
print("female words:")
for i in index_f:
    print(embedding_300_vocab[i])

In [40]:
male_most_biased = [embedding_300_vocab[i] for i in index_m]
female_most_biased = [embedding_300_vocab[i] for i in index_f]

In [42]:
def double_hard_debias(words, males, females):
    """Double Hard Debias:
    
    words: word embeddings of some corpus
    males: set of most biased male words 
    females: set of most biased female words
    """
    
    #need: Word embeddings, top 500 Male biased words set Wm, top 500 Female biased words set Wf
    #1. for all word embeddings: decentralize all words
    mue = (len(words)**(-1)) * np.sum(words, axis=0)
    # print(mue)
    words_decen = np.zeros((words.shape))
    for index, embedding in enumerate(words):
        # print(index,":",embedding)
        words_decen[index] = embedding - mue
    
    #print("decentralized:",words_decen)
    #print("origin:",words)
        
    #2. for all decentralized embeddings: compute PCA
    #princ_comp = np.asarray(pca_tft(words_decen))
    #print("Principal Components:",princ_comp)
    pca = PCA().fit(words_decen)
    princ_comp = pca.components_

    #print("Sklearn PC:", pca.components_)

    evaluations = []

    #3. for all principal components:
    for pc in princ_comp:
        male_proj = np.zeros((males.shape))
        male_debias = np.zeros((males.shape))
        female_proj = np.zeros((females.shape))
        female_debias = np.zeros((females.shape))
   
        for index, male in enumerate(males):
        #male embedding = decentralized embedding - projected original (?) embedding into direction of PC
            male_proj[index] = (male - mue) - ((np.transpose(pc)*male)*pc)
            #with all new male embeddings: HardDebias
            male_debias[index] = hardDebias(male_proj[index])
        
        for index, female in enumerate(females):
        #female embedding = decentralized embedding - projected original (?) embedding into direction of PC
            female_proj[index] = (female - mue) - ((np.transpose(pc)*male)*pc)
            #with all new female embeddings: HardDebias
            female_debias[index] = hardDebias(female_proj[index])
    
        #for all HardDebiased embeddings: KMeansClustering (2)
        #for clustered embeddings: compute gender alignment accuracy
        #4. store evaluations for each principal components
        evaluations.append(align_acc(male_debias, female_debias))
    
    #5. evaluate which PC lead to most random cluster (evaluation smallest (close to 0.5), used second PC)
    best_eval = evaluations.index(np.min(evaluations))
    best_pc = princ_comp[best_eval]
    #print("Best PC:",best_pc,"with evaluation:",evaluations[best_eval])

    first_debias = np.zeros((words.shape))
    #6. for all decentralized embeddings: remove that PC-direction
    for index,word in enumerate(words_decen):
        first_debias[index] = word - ((np.transpose(best_pc)*words[index])*best_pc)
    
    #7. for all new embeddings: HardDebias
    double_debias = np.zeros((words.shape))
    for index,word in enumerate(first_debias):
        double_debias[index] = hardDebias(word)

    return double_debias


In [43]:
#Gender alignment accuracy/ Neighborhood Metric:
def align_acc(males, females):
    """bias measurement using KMeans Clustering
    
    takes female and male word's embeddings
    ground truth labels:
    0 = male,
    1 = female"""
    
    #need: k (=1000) most biased female and male word's embedding (cosine similarity embedding & gender direction),
    #1. assign ground truth gender labels: 0 = male, 1 = female
    #2. run KMeans on embeddings
    kmeans = KMeans(n_clusters=2).fit(np.concatenate((males, females)))
    split = males.shape[0]
    correct = 0
    #print(kmeans.labels_)
    #3. compute alignment score: cluster assignment vs ground truth gender label
    for i in range(np.concatenate((males, females)).shape[0]):
        if i < split and kmeans.labels_[i] == 0:
            correct+= 1
        elif i >= split and kmeans.labels_[i] == 1:
            correct += 1
    
    #4. alignment score = max(a, 1-a)
    alignment = 1/(2*2) * correct
    alignment = np.maximum(alignment, 1-alignment)
    
    return alignment 

In [44]:
def hard_debias (N, equalize_pairs=equalize_pairs, embedding=embedding_300_word_vector, B=B):
    for i, e_pair in enumerate (equalize_pairs):
        for word in e_pair:
            mean = word/len(e_pair)
            simple_average_v = mean - mean_B
            w_embedding = simple_average_v + np.sqrt(1-v^2) * ((embedding_B - mean_B)/(embedding_B - mean_B)) #normed, v should also be normed (||v||)
    return B, w_embedding